# Experiments with the google API

### With help from the following tutorial(s):
- https://medium.com/google-cloud/sentiment-analysis-using-google-cloud-machine-learning-552be9b9c39b


### Along with the following documentation(s):
- https://cloud.google.com/natural-language/docs/reference/libraries#client-libraries-usage-python\
- https://developer.twitter.com/en/docs/tutorials/how-to-analyze-the-sentiment-of-your-own-tweets


# Imports/Download

### google-cloud-language library downloaded via the console with pip3 install --upgrade google-cloud-language and yaml via pip3 install pyyaml

In [31]:
import os
import requests
from google.cloud import language_v1
import yaml
import json


# Setting up Authentication for Google API

### Mostly done online on the google cloud dashboard. Created service account. Then set the key as an environment variable:

In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/Nanna/Desktop/EC601/google_keys/sentiment-analysis-test-327412-c89ef3ef857c.json"

# A simple query with custom test

## Preparation

### Instantiate a client object

In [5]:
client = language_v1.LanguageServiceClient()

### Generate text to analyze and make it a json document in the right format

In [9]:
#Text 
text = "Hello world! You are beautiful."

#create json document, content is text, document type plain text or html
doc = language_v1.Document(content = text, type_= language_v1.Document.Type.PLAIN_TEXT)

In [7]:
help(language_v1.Document)

Help on class Document in module google.cloud.language_v1.types.language_service:

class Document(proto.message.Message)
 |  Document(mapping=None, *, ignore_unknown_fields=False, **kwargs)
 |  
 |  Represents the input to API methods.
 |  
 |  Attributes:
 |      type_ (google.cloud.language_v1.types.Document.Type):
 |          Required. If the type is not set or is ``TYPE_UNSPECIFIED``,
 |          returns an ``INVALID_ARGUMENT`` error.
 |      content (str):
 |          The content of the input in string format.
 |          Cloud audit logging exempt since it is based on
 |          user data.
 |      gcs_content_uri (str):
 |          The Google Cloud Storage URI where the file content is
 |          located. This URI must be of the form:
 |          gs://bucket_name/object_name. For more details, see
 |          https://cloud.google.com/storage/docs/reference-uris. NOTE:
 |          Cloud Storage object versioning is not supported.
 |      language (str):
 |          The language 

## Request for analysis 

### Request passed and sentiment analysis of the sentence gotten and printed

In [16]:
sentiment_analysis = client.analyze_sentiment(request ={'document': doc})

In [17]:
print(type(sentiment_analysis))

<class 'google.cloud.language_v1.types.language_service.AnalyzeSentimentResponse'>


In [25]:
print("Full text: {}".format(text))
print()
print("Overall sentiment of document:\n{}".format(sentiment_analysis.document_sentiment))
print("Individual sentence scores:\n{}".format(sentiment_analysis.sentences))

Full text: Hello world! You are beautiful.

Overall sentiment of document:
magnitude: 1.7000000476837158
score: 0.800000011920929

Individual sentence scores:
[text {
  content: "Hello world!"
  begin_offset: -1
}
sentiment {
  magnitude: 0.699999988079071
  score: 0.699999988079071
}
, text {
  content: "You are beautiful."
  begin_offset: -1
}
sentiment {
  magnitude: 0.8999999761581421
  score: 0.8999999761581421
}
]


### Inspecting individual sentences test

In [30]:
first = sentiment_analysis.sentences[0]
print("Sentiment of first sentence\nContent: {}\nMagnitude: {}\nScore: {}".format(first.text.content, first.sentiment.magnitude, first.sentiment.score ))

Sentiment of first sentence
Content: Hello world!
Magnitude: 0.699999988079071
Score: 0.699999988079071


# Getting a tweets and passing them to google for analyzing the sentiment

### Testing future workflow for the project

## Manage Twitter API Credentials

### Create .yaml file for bearer instead of environment file and read credentials in. Create header object for url

In [44]:
def get_credentials():
    with open("/Users/Nanna/Desktop/EC601/twitter_sentiment_analysis/creds.yaml") as file:
        return yaml.safe_load(file)

def create_headers(bearer_token):
    headers = {}
    headers["Authorization"] = "Bearer {}".format(bearer_token)
    return headers

In [45]:
#initialise bearer token and create a header.
creds = get_credentials()
bearer_token = creds['twitter_api']['bearer_token']
headers = create_headers(bearer_token)

## Create URL for Twitter request
### Test query: tweets on trump

In [46]:
#create url
    #query Trump lang:en
def create_request():
    url = 'https://api.twitter.com/2/tweets/search/recent?query=Trump%20lang%3Aen'
    parameters = {
            'start_time': {},
            'end_time': {},
            'max_results' : 10,
            'expansions' : 'attachments.media_keys',
            'tweet.fields': 'created_at,public_metrics',
            'next_token': {}
        }
    return (url, parameters)

def connect_to_endpoint(url, headers, params = None):
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

## Connect to twitter and inspect tweets recieved

In [47]:
url, params = create_request()
twitter_response = connect_to_endpoint(url, headers, params)

Endpoint Response Code: 200


In [50]:
print(json.dumps(twitter_response['data'], indent= 4, sort_keys=True))

[
    {
        "created_at": "2021-09-30T01:35:18.000Z",
        "id": "1443388957044129798",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 82
        },
        "text": "RT @TayFromCA: Pam Bondi, the very talented and honorable frmr AG of FL, has our complete faith and confidence in taking over MAGA Action.\u2026"
    },
    {
        "attachments": {
            "media_keys": [
                "7_1443388515157430273"
            ]
        },
        "created_at": "2021-09-30T01:35:18.000Z",
        "id": "1443388957023211522",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 0
        },
        "text": "This video message is for Donald Trump and Fox news maybe this will help with those cry baby attitudes.\nUnited States Diplomat\nLarry Kuchenmeister https://t.co/086Zzkyl27"
    },
    {
 

### Extracting just text for sentiment analysis test

In [56]:
print(json.dumps(twitter_response['data'][0]['text'], indent= 4, sort_keys=True))
print()
print(len(twitter_response['data']))

"RT @TayFromCA: Pam Bondi, the very talented and honorable frmr AG of FL, has our complete faith and confidence in taking over MAGA Action.\u2026"

10


## Analyzing the sentiment of the Trump tweets 

### Create dummy function to check if score is positive or negative

In [62]:
def sentiment(score):
    s = float(score)
    if s > 0:
        return "positive"
    elif s == 0:
        return "neutral"
    else:
        return "negative"

## For loop that makes requests to the api and  to analyzes all the tweets

In [63]:
#do a loop to analyze all the tweets

for tweet in twitter_response['data']:
    text = tweet['text']
    doc = language_v1.Document(content = text, type_= language_v1.Document.Type.PLAIN_TEXT)
    sentiment_analysis = client.analyze_sentiment(request ={'document': doc})
    mag = sentiment_analysis.document_sentiment.magnitude
    score = sentiment_analysis.document_sentiment.score
    print("Tweet: {}".format(text))
    print("Tweet sentiment magnitude: {}, score: {}".format(mag, score))
    print("Overall feeling: {}".format(sentiment(score)))
    print()

Tweet: RT @TayFromCA: Pam Bondi, the very talented and honorable frmr AG of FL, has our complete faith and confidence in taking over MAGA Action.…
Tweet sentiment magnitude: 0.5, score: 0.5
Overall feeling: positive

Tweet: This video message is for Donald Trump and Fox news maybe this will help with those cry baby attitudes.
United States Diplomat
Larry Kuchenmeister https://t.co/086Zzkyl27
Tweet sentiment magnitude: 0.4000000059604645, score: 0.10000000149011612
Overall feeling: positive

Tweet: RT @jennycohn1: “But Trump’s election lies do not justify ignoring legitimate concerns about election security. Nor do they provide a free…
Tweet sentiment magnitude: 1.600000023841858, score: -0.800000011920929
Overall feeling: negative

Tweet: @0rdb9 @Drealstudmuffin @earthing5000 @timfisher836 @laura_7771 @Erica7016 @Mrcy_Grce @KeysLiisa @aingelgrl @ReallyKW @smrose29 @WenMaMa2 @JayC1l @ImaP91 @SPR2021 @Vet4DJT @txark76 @RDog861 @SunIslandMusic You are very welcome, Trump Supporter
@0rdb9


# Conlcusion
### As this was experimentation I allowed myself a rather free format. For the project I see I need to find a good workflow for the requests and keep things organized. The tweets might need some preprocessing, make sure the query is robust and make the sensitivity scale more detailed. Also, the sentiment analysis might be better to do in a single call to the api instead of n calls for n tweets